In [17]:
#import import_ipynb
#from Main import Board, Coord
from collections import namedtuple

Coord = namedtuple("Coord", ["X", "Y"]) 


In [25]:
import random
class Piece:
    """A class to define the individual pieces, their descriptions, type, shape, orientation, and color"""
    
    IDGenerator = list(range(9001, 9200))
    
    
    def __init__(self, pShape, pType="None", pOri="1", pColor="BLUE", pDesc=""): #idk if there's a purpose to the p's
        
        self.shape = [Coord(square.X, -square.Y) for square in pShape] #we mirror the shape since 
                            #representation when printing is flipped. the rotate function is also changed accordingly.
        self.type = "Basic"
        self.ori = pOri
        self.desc = pDesc
        self.color = pColor
        self.rotations = {1:self.shape}
        self.coordinates = []
        self.__CacheRotations__()
        
        self.upgraded = False
        
        #print(self.rotations)
        
        
        self.ID = self.__AssignID__()
        
        
    #def __iter__(self):
    #    return self
    # 
    #def __next__(self):
    def __getitem__(self, index):
        return self.shape[index]
    
    def __CacheRotations__(self):
        for ori in range(2, 5):
            self.rotations[ori] = self.Rotate()
        self.SetOrientation(1)
        
    
    @classmethod
    def __AssignID__(cls):
        newID = random.choice(cls.IDGenerator)
        cls.IDGenerator.remove(newID)
        return newID
    
    @classmethod
    def ResetAllPieceIDs(cls):
        """Reset the list of ID's that new pieces are picked from."""
        cls.IDGenerator = list(range(9001, 9200))
        #need to also clear all existing pieces probably
        
    
    def GetID(self):
        return self.ID
    
    def GetColor(self):
        return self.color
    
    def SetDescription(self, pDesc):
        self.desc = pDesc
        
    def GetOrientation(self):
        return self.ori
    
    def SetOrientation(self, ori):
        self.ori = ori
        self.shape = self.rotations[ori]
        
    def GetCoordinates(self):
        return self.coordinates
                
    def SetCoordinates(self, listCoord):
        self.coordinates = listCoord
        
    def Rotate(self):
        """Rotate the mirrored piece coordinates 90deg clockwise. To get a true 90d rotate:newCoord = Coord(oldCoord.Y, -oldCoord.X) """
        newRotation = []
        for oldCoord in self.shape:
            newCoord = Coord(-oldCoord.Y, oldCoord.X)
            newRotation.append(newCoord)
            
        self.shape = newRotation
        return newRotation
    
    
    #####Point Based Functions Next####
    def Activate(self, value, prevProcessList, nextProcessList):
        """Given a number value, the previously processed pieces in the active line, 
           and the upcoming pieces in the active line, calculate and return a new value."""
        
        
        print(prevProcessList, self.ID, nextProcessList)
        #return value + ((self.ID - 9000)//100) 
        
        if self.ID in prevProcessList:
            return value
        
        else:
            return value + 10
        
         

In [24]:
def Override(f):
    return f


class GreenPiece(Piece):
    """Extends our basic Piece class, giving it a color:GREEN, type:MULTIPLIER, and changes the activate fn."""
    def __init__(self, pShape, pType="None", pOri="1", pColor="BLUE", pDesc="", pMultiplier=2): #idk if there's a purpose to the p's
        
        self.shape = [Coord(square.X, -square.Y) for square in pShape] #we mirror the shape since 
                            #representation when printing is flipped. the rotate function is also changed accordingly.
        self.type = "MULTIPLIER"
        self.pMultiplier = pMultiplier
        self.ori = pOri
        self.desc = pDesc
        self.color = "GREEN"
        self.rotations = {1:self.shape}
        self.coordinates = []
        
        self.upgraded = False
        
        self.__CacheRotations__()
        #print(self.rotations)
        
        
        self.ID = self.__AssignID__()
        
    @Override #doesn't actually check anything, just pretty.
    def Activate(self, value, prevProcessList, nextProcessList):
        """Given a number value, multiplies the value by this piece's designated amount and returns the new value"""
        
        print(prevProcessList, self.ID, nextProcessList)
        
        if not self.upgraded:
            if self.ID in prevProcessList:
                return value
            else:
                return value * self.pMultiplier
        elif self.upgraded:
            #Compound multiplication for an upgraded piece, eg. if this piece covers two spots on the 
            #active line, then it multiplies twice.
            return value * self.pMultiplier